In [1]:
from utils.ArticlesHandler import ArticlesHandler
from utils import solve, embedding_matrix_2_kNN, get_rate, accuracy, precision, recall, f1_score
from utils import Config
import time
import numpy as np
import scipy.sparse as sp
from postprocessing.SelectLabelsPostprocessor import SelectLabelsPostprocessor
from pygcn.utils import encode_onehot, accuracy
from model import AGNN
import torch
import torch.nn.functional as F
import torch.optim as optim

Import config file and check some values

In [2]:
config = Config(file='config')

assert (config.num_fake_articles + config.num_real_articles > 
        config.num_nearest_neighbours), "Can't have more neighbours than nodes!"

print("Method of decomposition:", config.method_decomposition_embedding)

Method of decomposition: parafac


Import the articles and decompose the tensor.

In [3]:
print("Loading dataset", config.dataset_name)
articles = ArticlesHandler(config)

print("Performing decomposition...")
C = articles.get_tensor()

Loading dataset Random Poltical News Dataset
Performing decomposition...


/media/benamira/19793564030D4273/MCsBackup/3A/OMA/Projet/misinformation-detection-tensor-embeddings/tensorly/decomposition/_tucker.py:60: Warning: Given only one int for 'rank' intead of a list of 3 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


Get the labels

In [4]:
config.set("num_unknown_labels", 195)

In [5]:
labels = articles.articles.labels
all_labels = articles.articles.labels_untouched

In [6]:
index_higest_similarities = np.argpartition(C, 5, axis=1)[:, :5]
adj_mat = np.zeros((150, 150), dtype=int)
for i, neightbors in enumerate(index_higest_similarities):
    adj_mat[i, neightbors] = 1
# Force symetric
adj_mat = adj_mat + np.transpose(adj_mat)
adj_mat[adj_mat == 2] = 1

In [7]:
adj = torch.FloatTensor(np.array(adj_mat))
features = torch.FloatTensor(np.array(C))
all_labels = torch.LongTensor(all_labels)




In [8]:
#adj, features, all_labels = load_from_features(C, all_labels, config)
#_, _, labels = load_from_features(C, labels, config)

In [9]:
# idx_train = range(150)
# idx_val = range(150, 175)
# idx_test = range(175, 200)
print(labels)
idx_train = np.where(all_labels)[0]
idx_val = np.where(1 - abs(all_labels))[0][:90]
idx_test = np.where(1 - abs(all_labels))[0][90:]

print(len(idx_train))

idx_test = torch.LongTensor(np.array(idx_test))

idx_val = torch.LongTensor(np.array(idx_val))

idx_train = torch.LongTensor(np.array(all_labels))



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1]
150


In [10]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.nn.functional as F


class GraphAttentionLayer(nn.Module):

    def __init__(self, requires_grad=True):
        super(GraphAttentionLayer, self).__init__()
        if requires_grad:
            # unifrom initialization
            self.beta = Parameter(torch.Tensor(1).uniform_(
                0, 1), requires_grad=requires_grad)
        else:
            self.beta = Variable(torch.zeros(1), requires_grad=requires_grad)

    def forward(self, x, adj):


        # NaN grad bug fixed at pytorch 0.3. Release note:
        #     `when torch.norm returned 0.0, the gradient was NaN.
        #     We now use the subgradient at 0.0, so the gradient is 0.0.`
        norm2 = torch.norm(x, 2, 1).view(-1, 1)

        # add a minor constant (1e-7) to denominator to prevent division by
        # zero error
        if torch.cuda.is_available():
            cos = self.beta.cuda() * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)
        else:
            cos = self.beta * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)

        # neighborhood masking (inspired by this repo:
        # https://github.com/danielegrattarola/keras-gat)
        mask = (1. - adj) * -1e9
        masked = cos + mask

        # propagation matrix
        P = F.softmax(masked, dim=1)

        # attention-guided propagation
        output = torch.mm(P, x)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (16 -> 16)'


class LinearLayer(nn.Module):

    def __init__(self, in_features, out_features, initializer=nn.init.xavier_uniform_):
        super(LinearLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(initializer(
            torch.Tensor(in_features, out_features)))

    def forward(self, input):
        # no bias
        return torch.mm(input, self.weight)

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'


class AGNN(nn.Module):

    def __init__(self, nfeat, nhid, nclass, nlayers, dropout_rate):
        super(AGNN, self).__init__()

        self.layers = nlayers
        self.dropout_rate = dropout_rate

        self.embeddinglayer = LinearLayer(nfeat, nhid)
        nn.init.xavier_uniform_(self.embeddinglayer.weight)

        self.attentionlayers = nn.ModuleList()
        # for Cora dataset, the first propagation layer is non-trainable
        # and beta is fixed at 0
        self.attentionlayers.append(GraphAttentionLayer(requires_grad=False).cuda())
        for i in range(1, self.layers):
            if torch.cuda.is_available():
                self.attentionlayers.append(GraphAttentionLayer().cuda())
            else:
                self.attentionlayers.append(GraphAttentionLayer())


        self.outputlayer = LinearLayer(nhid, nclass)
        nn.init.xavier_uniform_(self.outputlayer.weight)

    def forward(self, x, adj):
        x = F.relu(self.embeddinglayer(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)

        for i in range(self.layers):
            x = self.attentionlayers[i](x, adj)

        x = self.outputlayer(x)
        x = F.dropout(x, self.dropout_rate, training=self.training)
        return F.log_softmax(x, dim=1)

In [11]:
cuda = True
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
fastmode = False
epochs = 150
layers =4

# Model and optimizer
model = AGNN(nfeat=features.shape[1],
                     nhid=hidden,
                     nclass=2,
                     nlayers=layers,
                     dropout_rate=0.5)

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    all_labels = all_labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    
    print(all_labels[idx_train])
    
    loss_train = F.nll_loss(output[idx_train], all_labels[idx_train])
    acc_train = accuracy(output[idx_train], all_labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], all_labels[idx_val])
    acc_val = accuracy(output[idx_val], all_labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], all_labels[idx_test])
    acc_test = accuracy(output[idx_test], all_labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

tensor([ 1,  1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1,  1,  1,
         1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1,  1, -1,
        -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
        -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1, -1,
         1, -1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,
        -1, -1,  1, -1,  1, -1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1,
        -1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,
         1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,
        -1, -1, -1, -1,  1, -1], device='cuda:0')


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorMath.cu:24